In [1]:
import pandas as pd
from string import punctuation
import re
import numpy as np
from setup import *
import glob
import os
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

Using TensorFlow backend.


# The Process

## Data Acquisition 

1. Use Glob to collect all of the files in the Data Folder
1. Save to single file for later use

## Preprocessing

1. Collect the columns for reach, engagement
1. Create proportion column
1. Make Variable for how much reach to keep
1. Tokenize
1. Tf-Idf


## Save Preprocessing Steps
1. Save teh Tf-Idf using pickle
1. Condense the preprocessing step into a function

## Model Setup
1. Support Vector Machine Regression
1. test-train split

## Train The Model, Tune Hyperparameters
1. Gridsearch on some parameters (maybe include how much reach to give to model)
1. Save all of the models, compare outputs, etc
1. Cross validate

## Test Model
1. Test each model on the testing set, look for best results. 

## Save Best Model 
1. Use pickle to save the best model for future use. 